In [135]:
import pandas as pd
df1 = pd.read_csv("onsen_info.csv")

def filter_df(df, lat, long):
    threshold = 1
    while(1):
        print(df["東経(゜)"])
        if len(df[(df["東経(゜)"] + 0.01*df["東経(′)"] + 0.0001*df["東経(″)"] - long)**2 + (df["北緯(゜)"] + 0.01*df["北緯(′)"] + 0.0001*df["北緯(″)"] - lat)**2 <= threshold]) == 1:
            break
        else:
            threshold = threshold - 0.1
    return df[(df["東経(゜)"] + 0.01*df["東経(′)"] + 0.0001*df["東経(″)"] - long)**2 + (df["北緯(゜)"] + 0.01*df["北緯(′)"] + 0.0001*df["北緯(″)"] - lat)**2 <= threshold+0.1]

df_test = pd.read_csv("./suisitucsv/愛知.csv")
columns = []
for item in df1.columns:
    columns.append(item)
for item in df_test.columns:
    columns.append(item)
df = pd.DataFrame(columns=columns)

for i, row in df1.iterrows():
    print(row)
    if row.latitude != "不明":
        pref = row.prefecture
        city = row.city
        name = row.name
        lat = float(row.latitude)
        long = float(row.longitude)
        if pref[-1] != "道":
            pref = pref[:-1]
            df2 = pd.read_csv("./suisitucsv/"+pref+".csv")
        else:
            df2 = pd.concat([pd.read_csv("./suisitucsv/"+pref+"1.csv"), pd.read_csv("./suisitucsv/"+pref+"2.csv")])
        df2 = df2.dropna(subset=["東経(゜)", "東経(′)", "東経(″)", "北緯(゜)", "北緯(′)", "北緯(″)"])
        
        
        min_dis = 10000
        nearest_data = pd.Series()
        for index, data in df2.iterrows():
            try:
                cur_dis = (data["東経(゜)"] + 0.01 * data["東経(′)"] + 0.0001 * data["東経(″)"] - long) ** 2 + (data["北緯(゜)"] + 0.01 * data["北緯(′)"] + 0.0001 * data["北緯(″)"] - lat) ** 2
                if cur_dis < min_dis:
                    min_dis = cur_dis
                    nearest_data = data
            except TypeError:
                pass
        data = pd.concat([row, nearest_data])
        df = df.append(data, ignore_index=True)
df.to_csv("merged_onsen_info.csv", index=False)
            
                    



name                                    箱根温泉
prefecture                              神奈川県
city                                     箱根町
lv01Nm                                   仙石原
link          http://www.hakone-ryokan.or.jp
hyoka                                   4.22
comi                                   78705
yado                                     234
onsen_num                                212
dai                                      150
roten                                    146
sauna                                     39
kinen                                    205
kokyu                                     45
tanjun                                  True
enka                                    True
tansan                                  True
iou                                     True
housya                                  True
ryusan                                  True
sansei                                  True
gantetsu                               False
nisan     

<ipython-input-135-c50095529dd7>:39: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  nearest_data = pd.Series()
<ipython-input-135-c50095529dd7>:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
<ipython-input-135-c50095529dd7>:39: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  nearest_data = pd.Series()
<ipython-input-135-c50095529dd7>:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
<ipython-input-135-c50095529dd7>:49: FutureWarning: In a future version, object-dtype columns with all-bool values will no

name                             那須温泉郷
prefecture                         栃木県
city                               那須町
lv01Nm                           大字高久丙
link          http://www.nasuonsen.com
hyoka                             4.19
comi                             35195
yado                               113
onsen_num                          101
dai                                 45
roten                               61
sauna                               14
kinen                              101
kokyu                                9
tanjun                            True
enka                              True
tansan                            True
iou                               True
housya                            True
ryusan                            True
sansei                            True
gantetsu                         False
nisan                            False
youso                            False
latitude              37.0834479467773
longitude              14

<ipython-input-135-c50095529dd7>:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
<ipython-input-135-c50095529dd7>:49: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df = df.append(data, ignore_index=True)
<ipython-input-135-c50095529dd7>:39: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  nearest_data = pd.Series()
<ipython-input-135-c50095529dd7>:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
<ipython-input-135-c50095529dd7>:49: FutureWarning: In a future version, object-dtype columns wit

name                                 湯田中渋温泉郷
prefecture                               長野県
city                                    山ノ内町
lv01Nm                                  大字平穏
link          http://www.yudanaka-onsen.info
hyoka                                   4.33
comi                                   11441
yado                                      75
onsen_num                                 65
dai                                       39
roten                                     33
sauna                                      7
kinen                                     61
kokyu                                      5
tanjun                                  True
enka                                    True
tansan                                  True
iou                                     True
housya                                  True
ryusan                                  True
sansei                                 False
gantetsu                                True
nisan     

...

name                鶴巻温泉
prefecture          神奈川県
city                 秦野市
lv01Nm            鶴巻北二丁目
link                  なし
hyoka                  ―
comi                   ―
yado                   2
onsen_num              0
dai                    0
roten                  0
sauna                  0
kinen                  0
kokyu                  0
tanjun             False
enka                True
tansan             False
iou                False
housya             False
ryusan             False
sansei             False
gantetsu           False
nisan              False
youso              False
latitude       35.382596
longitude     139.276333
Name: 601, dtype: object
name           底倉温泉
prefecture     神奈川県
city             不明
lv01Nm           不明
link             なし
hyoka             ―
comi              ―
yado              1
onsen_num         0
dai               0
roten             0
sauna             0
kinen             0
kokyu             0
tanjun        False
enka           True
tansa

<ipython-input-135-c50095529dd7>:39: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  nearest_data = pd.Series()
<ipython-input-135-c50095529dd7>:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
<ipython-input-135-c50095529dd7>:49: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df = df.append(data, ignore_index=True)
<ipython-input-135-c50095529dd7>:39: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  nearest_data = pd.Series()
<ipython-input-135-c50095529dd7>:49: FutureWarning: The frame.append method is dep

name                                                      接岨峡温泉
prefecture                                                  静岡県
city                                                         不明
lv01Nm                                                       不明
link          http://www.okuooi.gr.jp/wordpress/onsen/o_sess...
hyoka                                                         ―
comi                                                          ―
yado                                                          0
onsen_num                                                     0
dai                                                           0
roten                                                         0
sauna                                                         0
kinen                                                         0
kokyu                                                         0
tanjun                                                    False
enka                                    

name           柴石温泉
prefecture      大分県
city             不明
lv01Nm           不明
link             なし
hyoka             ―
comi              ―
yado              1
onsen_num         0
dai               0
roten             0
sauna             0
kinen             0
kokyu             0
tanjun         True
enka           True
tansan        False
iou           False
housya        False
ryusan         True
sansei        False
gantetsu      False
nisan         False
youso         False
latitude         不明
longitude        不明
Name: 624, dtype: object
name           川汲温泉
prefecture      北海道
city             不明
lv01Nm           不明
link             なし
hyoka             ―
comi              ―
yado              1
onsen_num         0
dai               0
roten             0
sauna             0
kinen             0
kokyu             0
tanjun         True
enka          False
tansan        False
iou           False
housya        False
ryusan        False
sansei        False
gantetsu      False
nisan         F

<ipython-input-135-c50095529dd7>:39: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  nearest_data = pd.Series()
<ipython-input-135-c50095529dd7>:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
<ipython-input-135-c50095529dd7>:49: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df = df.append(data, ignore_index=True)
